In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

### Normalize action between -1 to 1, save min, max info to the hdf5 to be used during inference.

In [2]:
# dataset_path = "/home/ns1254/data_robomimic/lift/ns/lift_image81_norm.hdf5"
# dataset_path = "/home/ns1254/data_robomimic/lift/ns/lift_image81.hdf5" # no normalization

dataset_path = "/home/ns1254/data_robomimic/lift/ns/lift_image141_norm.hdf5"

In [3]:
f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())

In [4]:
lengths=[]
demos_minmax={}
for demo_name in demos:
    demo=f['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

    action=f['data'][demo_name]['actions']
    action=np.array(action) 
    demos_minmax[demo_name] = (np.min(action, axis=0), np.max(action, axis=0))


lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  141
Max length:  667
Min length:  129
Mean length:  199.60283687943263


In [5]:
mins,maxs=[],[]
for demo_name in demos_minmax.keys():
    min,max= demos_minmax[demo_name]
    mins.append(min)
    maxs.append(max)

mins=np.min(mins, axis=0)
maxs=np.max(maxs, axis=0) + 1e-8
mins.shape, maxs.shape

((7,), (7,))

In [6]:
mins

array([ -1.11913306,  -1.61284268, -13.44231429,   0.        ,
         0.        ,  -0.        ,  -1.        ])

In [7]:
maxs

array([3.65146826e+00, 9.74293600e-01, 6.97497747e+00, 1.00000000e-08,
       1.00000000e-08, 1.00000000e-08, 1.00000001e+00])

In [8]:
# for demo_name in demos:
#     demo=f['data'][demo_name]  
#     action=f['data'][demo_name]['actions']
#     # convert action to [-1,1] using mins and maxs
#     action= -1 + ( (action-mins)/(maxs-mins) )* 2.0

#     del f["data"][demo_name]['actions']
#     f["data"][demo_name].create_dataset('actions', data=action)

In [ ]:
# save stats to f as another key 
f.create_dataset('mins', data=mins)
f.create_dataset('maxs', data=maxs)

In [9]:
# f.close()

In [5]:
f.keys()

<KeysViewHDF5 ['data', 'mask', 'maxs', 'mins']>

In [6]:
for key in f['mask'].keys():
    print(key, f['mask'][key])

bad <HDF5 dataset "bad": shape (22,), type "|S8">
good <HDF5 dataset "good": shape (45,), type "|S8">
goodbad <HDF5 dataset "goodbad": shape (67,), type "|S8">
goodbad10 <HDF5 dataset "goodbad10": shape (55,), type "|S8">
goodbad15 <HDF5 dataset "goodbad15": shape (60,), type "|S8">
goodbad5 <HDF5 dataset "goodbad5": shape (50,), type "|S8">
goodgray <HDF5 dataset "goodgray": shape (59,), type "|S8">
goodgray10 <HDF5 dataset "goodgray10": shape (55,), type "|S8">
goodgray5 <HDF5 dataset "goodgray5": shape (50,), type "|S8">
gray <HDF5 dataset "gray": shape (14,), type "|S8">
train <HDF5 dataset "train": shape (73,), type "|S8">
valid <HDF5 dataset "valid": shape (8,), type "|S8">


In [16]:
# for key in f['mask'].keys():
#     if key not in ['train' ,'valid']:
#         print(key, f['mask'][key])
#         del f['mask'][key]

In [8]:
# hdf5_path=dataset_path

# for key in f['mask'].keys():
#     demos=[b.decode('utf-8') for b in f['mask'][key] ]
    
#     del f['mask'][key]
    
#     filter_keys=sorted([elem for elem in demos])
#     filter_name=key
#     create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [11]:
# del f['mask']['good3']

In [15]:
good=[b.decode('utf-8') for b in f['mask']['good'] ]

In [16]:
# good=np.array(good, dtype='S8')
good

['demo_',
 'demo_',
 'demo_1',
 'demo_12',
 'demo_18',
 'demo_19',
 'demo_2',
 'demo_2',
 'demo_22',
 'demo_23',
 'demo_26',
 'demo_28',
 'demo_29',
 'demo_3',
 'demo_30',
 'demo_31',
 'demo_36',
 'demo_39',
 'demo_40',
 'demo_42',
 'demo_43',
 'demo_46',
 'demo_47',
 'demo_49',
 'demo_5',
 'demo_53',
 'demo_55',
 'demo_6',
 'demo_6',
 'demo_61',
 'demo_62',
 'demo_63',
 'demo_65',
 'demo_66',
 'demo_68',
 'demo_7',
 'demo_7',
 'demo_70',
 'demo_71',
 'demo_72',
 'demo_75',
 'demo_76',
 'demo_79',
 'demo_8',
 'demo_80']

In [7]:
f.close()

In [27]:
good=[b.decode('utf-8') for b in f['mask']['good'] ]
bad=[b.decode('utf-8') for b in f['mask']['bad'] ]
gray=[b.decode('utf-8') for b in f['mask']['gray'] ]

In [28]:
goodbad=good+bad
goodbad5=good+bad[:5]
goodbad10=good+bad[:10]
goodbad15=good+bad[:15]

goodgray=good+gray
goodgray5=good+gray[:5]
goodgray10=good+gray[:10]

goodbad=np.array(goodbad, dtype='S8')
goodbad5=np.array(goodbad5, dtype='S8')
goodbad10=np.array(goodbad10, dtype='S8')
goodbad15=np.array(goodbad15, dtype='S8')

goodgray=np.array(goodgray, dtype='S8')
goodgray5=np.array(goodgray5, dtype='S8')
goodgray10=np.array(goodgray10, dtype='S8')

In [29]:
hdf5_path=dataset_path

filter_keys=sorted([elem for elem in goodbad])
filter_name='goodbad'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)


filter_keys=sorted([elem for elem in goodbad5])
filter_name='goodbad5'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)



filter_keys=sorted([elem for elem in goodbad10])
filter_name='goodbad10'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)


filter_keys=sorted([elem for elem in goodbad15])
filter_name='goodbad15'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)


In [30]:
hdf5_path=dataset_path

filter_keys=sorted([elem for elem in goodgray])
filter_name='goodgray'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)


filter_keys=sorted([elem for elem in goodgray5])
filter_name='goodgray5'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)


filter_keys=sorted([elem for elem in goodgray10])
filter_name='goodgray10'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [31]:
f.close()

In [12]:
np.array(f['mins'])

array([-117.13325644, -137.97657021,  -30.3125    ,    0.        ,
          0.        ,  -30.        ,   -1.        ])

In [13]:
np.array(f['maxs'])

array([1.38379805e+02, 1.75085897e+02, 1.34375000e+01, 1.00000000e-08,
       1.00000000e-08, 6.50000000e+01, 1.00000001e+00])

In [24]:
# # save stats to f as another key
# dataset_path = "/home/ns1254/data_robomimic/lift/ns/lift_image81_norm.hdf5"
# f = h5py.File(dataset_path, "a")
# # f.create_dataset('stats', data=stats)
# f.create_dataset('mins', data=mins)
# f.create_dataset('maxs', data=maxs)
# f.close()

### create groups

In [18]:
names="""
demo_12.mp4
demo_14.mp4
demo_18.mp4
demo_19.mp4
demo_22.mp4
demo_23.mp4
demo_24.mp4
demo_26.mp4
demo_28.mp4
demo_29.mp4
demo_2.mp4
demo_30.mp4
demo_31.mp4
demo_34.mp4
demo_36.mp4
demo_39.mp4
demo_40.mp4
demo_42.mp4
demo_43.mp4
demo_44.mp4
demo_46.mp4
demo_47.mp4
demo_49.mp4
demo_4.mp4
demo_53.mp4
demo_54.mp4
demo_55.mp4
demo_61.mp4
demo_62.mp4
demo_63.mp4
demo_64.mp4
demo_65.mp4
demo_66.mp4
demo_68.mp4
demo_6.mp4
demo_70.mp4
demo_71.mp4
demo_72.mp4
demo_74.mp4
demo_75.mp4
demo_76.mp4
demo_79.mp4
demo_7.mp4
demo_80.mp4
demo_8.mp4
"""

In [19]:
demo_ids  = [line.replace(".mp4", "") for line in names.strip().split('\n')]
len(demo_ids), demo_ids[:5]

(45, ['demo_12', 'demo_14', 'demo_18', 'demo_19', 'demo_22'])

In [20]:
good_demos=demo_ids
good_demos=np.array(good_demos, dtype='S8')

hdf5_path=dataset_path

filter_keys=sorted([elem for elem in good_demos])
filter_name='good'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [22]:
names_bad="""
demo_10.mp4
demo_11.mp4
demo_13.mp4
demo_16.mp4
demo_17.mp4
demo_1.mp4
demo_20.mp4
demo_21.mp4
demo_25.mp4
demo_27.mp4
demo_35.mp4
demo_3.mp4
demo_41.mp4
demo_45.mp4
demo_51.mp4
demo_52.mp4
demo_56.mp4
demo_57.mp4
demo_59.mp4
demo_60.mp4
demo_67.mp4
demo_69.mp4
"""

demo_ids  = [line.replace(".mp4", "") for line in names_bad.strip().split('\n')]
len(demo_ids), demo_ids[:5]

(22, ['demo_10', 'demo_11', 'demo_13', 'demo_16', 'demo_17'])

In [23]:
bad_demos=demo_ids
bad_demos=np.array(bad_demos, dtype='S8')

hdf5_path=dataset_path

filter_keys=sorted([elem for elem in bad_demos])
filter_name='bad'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [24]:
names_bad="""
demo_15.mp4
demo_32.mp4
demo_33.mp4
demo_37.mp4
demo_38.mp4
demo_48.mp4
demo_50.mp4
demo_58.mp4
demo_5.mp4
demo_73.mp4
demo_77.mp4
demo_78.mp4
demo_81.mp4
demo_9.mp4
"""

demo_ids  = [line.replace(".mp4", "") for line in names_bad.strip().split('\n')]
len(demo_ids), demo_ids[:5]

(14, ['demo_15', 'demo_32', 'demo_33', 'demo_37', 'demo_38'])

In [25]:
gray_demos=demo_ids
gray_demos=np.array(gray_demos, dtype='S8')

hdf5_path=dataset_path

filter_keys=sorted([elem for elem in gray_demos])
filter_name='gray'
filter_lengths = create_hdf5_filter_key(hdf5_path=hdf5_path, demo_keys=filter_keys, key_name=filter_name)

In [14]:
f.close()

### verify

In [10]:
f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())
  
lengths=[]
for demo_name in demos:
    demo=f['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  81
Max length:  628
Min length:  84
Mean length:  223.07407407407408


In [11]:
ids_toclip=[]
for demo_name in demos:
    demo=f['data'][demo_name]
    actions=np.array(demo['actions'])
    if np.min(actions)<-1 or np.max(actions)>1:
        ids_toclip.append(demo_name)

print('Number of demos to clip: ', len(ids_toclip))

Number of demos to clip:  0


In [12]:
f.close()